In [1]:
from pprint import pprint
import pandas as pd
import numpy as np
import pyodbc as pc
import sqlalchemy
import warnings
import threading
from time import sleep

warnings.filterwarnings('ignore')

path_start = 'c:/tt/'
path_start_kntr = 'c:/tt/kntr/'

In [2]:
def connect():
    connection_string = "Driver=SQL Server;Server=localhost;Database=fabr;Trusted_Connection=Yes;"
    cnxn = pc.connect(connection_string, autocommit=True)
    return cnxn
def connect_sqlalchemy():
    engine = sqlalchemy.create_engine("mssql+pyodbc://DESKTOP-SFHJLGK/fabr?driver=ODBC+Driver+17+for+SQL+Server")
    return engine

engine = connect_sqlalchemy()

In [20]:
def mt_set_default():
    ds = {'M63': {
                '1': {
                    'rs1': '2ГП',
                    'rs2': '3-5',
                    'state1': 0,
                    'state2': 0,
                    'fl':0,
                    'end_idMt':-1
                },
                '2': {
                    'rs1': '3-5',
                    'rs2': '5/9П',
                    'state1': 0,
                    'state2': 0,
                    'fl':0,
                    'end_idMt':-1
                },
                '3': {
                    'rs1': '5/9П',
                    'rs2': '9',
                    'state1': 0,
                    'state2': 0,
                    'fl':0,
                    'end_idMt':10
                },
                '4': {
                    'rs1': '9',
                    'rs2': '11-19',
                    'state1': 0,
                    'state2': 0,
                    'fl':0,
                    'end_idMt':-1
                },
                '5': {
                    'rs1': '11-19',
                    'rs2': '23/29П',
                    'state1': 0,
                    'state2': 0,
                    'fl':0,
                    'end_idMt':-1
                },
                '6': {
                    'rs1': '23/29П',
                    'rs2': '',
                    'state1': 0,
                    'state2': 0,
                    'fl':0,
                    'end_idMt':15
                },
             }

     }
    return ds

def update_mt_rs_state(name_mt):
    err = ''
    ds1 = ds[name_mt]
    for key, s in ds1.items():
        rs1 = s['rs1']
        rs2 = s['rs2']
        fl = s['fl']
        st1 = df1[df1['name']==rs1]['state'].iloc[0]
        st2 = ''
        ds1[key]['state1'] = st1
        if rs2 != '':
            st2 = df1[df1['name']==rs2]['state'].iloc[0]
            ds1[key]['state2'] = st2
        err = chek_state_set_fl(ds1, key, st1, st2, fl)
        if err != '':
            return err
    return err


def chek_state_set_fl(ds1, key, st1, st2, fl):
    err = ''
    if st2 == '':
        if st1 == 1:
            if fl != 3:
                ds1[key]['fl'] = 2
            else:
                err = 'st1 == 1 fl != 3'
        if st1 == 0 and fl==2:
            ds1[key]['fl'] = 3
    else:
        if st1 == 1 and st2 == 0:
            if fl == 0 or fl == 1:
                ds1[key]['fl'] = 1
            else:
                err = 'st1 == 1 and st2 == 0 '
        if st1 == 1 and st2 == 1:
            if fl == 1 or fl == 2:
                ds1[key]['fl'] = 2
            else:
                err = 'st1 == 1 and st2 == 1'
        if st1 == 0 and st2 == 1:
            if fl == 2 or fl == 3:
                ds1[key]['fl'] = 3
            else:
                err = 'st1 == 0 and st2 == 1'
    return err

def chek_fl(name_mt):
    ds1 = ds[name_mt]

    for key, s in ds1.items():
        fl = s['fl']
        key_pred = str(int(key)-1)
        key_next = str(int(key)+1)
        fl_pred  = s.get(key_pred)
        fl_next = s.get(key_next)

        if fl_pred:
            if fl == 0  and fl_pred > 1:
                return 'fl == 0  and fl_pred > 1'
            if fl==1 and fl_pred != 2:
                return 'fl==1 and fl_pred != 2'
            if  fl==2 and fl_pred < 2:
                return 'fl==2 and fl_pred < 2'
            if fl==3 and fl_pred != 3:
                return 'fl==3 and fl_pred != 3'

        if fl_next:
            if (fl == 0 or fl==1) and fl_next != 0:
                return '(fl == 0 or fl==1) and fl_next != 0'
            if fl == 2 and (fl_next != 1 or fl_next != 2):
                return 'fl == 2 and (fl_next != 1 or fl_next != 2)'
            if fl==3 and (fl_next == 0):
                return 'fl==3 and (fl_next == 0)'
    return ''

def delete_mt_rs(name_mt):
    global del_mt
    ds1 = ds[name_mt]
    for key, s in ds1.items():
        fl = s['fl']
        if fl==3:
            del_mt = key
            ds1.pop(key, None)

def ff(name_mt):
    global  err1
    while True:
        ds1 = ds[name_mt]
        # if len(ds1)==0:
        #     err1 = 2
        #     break
        err = update_mt_rs_state(name_mt)
        if err == '':
            err = chek_fl(name_mt)
            if err == '':
                # delete_mt_rs(name_mt)
                pass
            else:
                err1 = err
                break
        else:
            err1 = err
            break

        last_key = str(len(ds[name_mt]))
        if ds[name_mt][last_key]['fl'] == 3:
            err1 = 'good'
            break
        sleep(1)


def f13(ls):
    # ls = [[0,0], [0,1], [1,1], [0,0], [2,1], [1,0], [3,1], [2,0], [4,1], [3,0], [4,0]]
    for ls1 in ls:
        df1.loc[df1['name']==ls1[0], 'state'] = ls1[1]
        sleep(3)

In [21]:
# ls1 = [[1,0], [1,1], [2,1],   [4,1],   [1,0], [2,0], [4,1], [3,0], [5,1], [4,0], [5,0]]
err1= 0
del_mt = ''
df1 = pd.read_sql('select * from rs', engine)
ds = mt_set_default()
df = pd.DataFrame(ds['M63'])
ls1 = [[1,0], [1,1], [2,1],    [1,0], [3,1], [2,0], [4,1], [3,0], [5,1], [4,0],  [6,1],[5,0], [6,0]]
ls = []
for sl in ls1:
    ls.append([df.loc['rs1',str(sl[0])], sl[1]])


t = threading.Thread(target=ff, args=('M63',))
t.start()

t1 = threading.Thread(target=f13, args=(ls,))
t1.start()

In [27]:
df = pd.DataFrame(ds['M63'])
df

,1,2,3,4,5,6
rs1,2ГП,3-5,5/9П,9,11-19,23/29П
rs2,3-5,5/9П,9,11-19,23/29П,
state1,0,0,0,0,0,0
state2,0,0,0,0,0,0
fl,3,3,3,3,3,3
end_idMt,-1,-1,10,-1,-1,15


In [6]:
ds = mt_set_default()
last_key = str(len(ds['M63']))

In [8]:
ds['M63'][last_key]['fl']

0

In [87]:
name_mt = 'M63'
ds1 = ds[name_mt]
if len(ds1)==0:
    err1 = 2
err = update_mt_rs_state(ds, name_mt)
if err == '0':
    err = chek_fl(ds, name_mt)
    if err == '0':
        delete_mt_rs(ds, name_mt)
    else:
        err1 = err
else:
    err1 = err

In [89]:
name_mt = 'M63'
ds1 = ds[name_mt]
if len(ds1)==0:
    err1 = 2

In [90]:
err = update_mt_rs_state(ds, name_mt)

In [92]:
err = chek_fl(ds, name_mt)

In [93]:
delete_mt_rs(ds, name_mt)

In [105]:
df1

,id,name,reg_rd,bit_rd,state,remont,marsh
0,1,1ГП,901,0,0,0,0
1,2,2ГП,901,1,0,0,0
2,3,1,901,2,0,0,0
3,4,1/15П,901,3,0,0,0
4,5,3-5,901,4,0,0,0
...,...,...,...,...,...,...,...
148,149,Л2,910,4,0,0,0
149,150,Л4,910,5,0,0,0
150,151,Л6,910,6,0,0,0
151,152,Л8,910,7,0,0,0


In [42]:
df = pd.DataFrame(ds1)
df

,1,2,3,4,5,6
rs1,2ГП,3-5,5/9П,9,11-19,23/29П
rs2,3-5,5/9П,9,11-19,23/29П,
state1,1,0,0,0,0,0
state2,0,0,0,0,0,0
fl,0,0,0,0,0,0
end_idMt,-1,-1,10,-1,-1,15


In [34]:
# df1[df1['name']==rs1].iloc[0]['state'] = 1
df1.loc[df1['name']==rs1, 'state'] = 1

In [35]:
df1

,id,name,reg_rd,bit_rd,state,remont,marsh
0,1,1ГП,901,0,0,0,0
1,2,2ГП,901,1,1,0,0
2,3,1,901,2,0,0,0
3,4,1/15П,901,3,0,0,0
4,5,3-5,901,4,0,0,0
...,...,...,...,...,...,...,...
148,149,Л2,910,4,0,0,0
149,150,Л4,910,5,0,0,0
150,151,Л6,910,6,0,0,0
151,152,Л8,910,7,0,0,0


In [33]:
rs1

'2ГП'